In [50]:
import numpy as np
N, C, H, W = 1, 1, 2, 2
input_data = np.ones([N, C, H, W])
print(input_data)

[[[[ 1.  1.]
   [ 1.  1.]]]]


In [51]:
pad = 1
filter_h = 2
filter_w = 2
stride = 1
out_h = (H + 2*pad - filter_h)//stride + 1
out_w = (W + 2*pad - filter_w)//stride + 1
print(out_h, out_w)

3 3


In [52]:
img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant', constant_values=(0, ))
print(img)

[[[[ 0.  0.  0.  0.]
   [ 0.  1.  1.  0.]
   [ 0.  1.  1.  0.]
   [ 0.  0.  0.  0.]]]]


In [53]:
col = np.zeros((N * C * out_h * out_w, C * filter_h * filter_w))
print(col)

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [ ]:
for n in range(N):
    for c in range(N):
        for h in range(out_h):
            for w in range(out_w):

In [43]:
col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))
print(col)

[[[[[[ 0.  0.  0.]
     [ 0.  0.  0.]
     [ 0.  0.  0.]]

    [[ 0.  0.  0.]
     [ 0.  0.  0.]
     [ 0.  0.  0.]]]


   [[[ 0.  0.  0.]
     [ 0.  0.  0.]
     [ 0.  0.  0.]]

    [[ 0.  0.  0.]
     [ 0.  0.  0.]
     [ 0.  0.  0.]]]]]]


In [44]:
for y in range(filter_h):
    y_max = y + stride * out_h      # y_max = y + 5
    for x in range(filter_w):
        x_max = x + stride * out_w  # x_max = x + 5
        col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
print(col)

[[[[[[ 0.  0.  0.]
     [ 0.  1.  1.]
     [ 0.  1.  1.]]

    [[ 0.  0.  0.]
     [ 1.  1.  0.]
     [ 1.  1.  0.]]]


   [[[ 0.  1.  1.]
     [ 0.  1.  1.]
     [ 0.  0.  0.]]

    [[ 1.  1.  0.]
     [ 1.  1.  0.]
     [ 0.  0.  0.]]]]]]


In [39]:
col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w, -1)
print(col)

[[ 0.  0.  0.  1.]
 [ 0.  0.  1.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  1.  0.  0.]
 [ 1.  0.  0.  0.]]


In [1]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """
    Parameters
    ----------
    input_data : 이미지 데이터
    filter_h : 필터 높이
    filter_w : 필터 폭
    stride : 스트라이드
    pad : 패드

    Returns
    -------
    col : 2차원행렬
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w, -1)
    return col

In [ ]:
import sys, os
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient

class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]